<a href="https://colab.research.google.com/github/ticky1515/Toxic-messages-analysis/blob/main/Deep_learning_USE_M(tokenized).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import

In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
%matplotlib inline
import seaborn as sns
import re

In [ ]:
!pip install Keras-Preprocessing

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 1.5 MB/s eta 0:00:00


In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
try:
    %tensorflow_version 2.x
except:
    pass

Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.


In [ ]:
# release version 
! pip install pythainlp

!pip install epitran
!pip install sklearn_crfsuite
!pip install tensorflow deepcut
!pip install attacut

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.4/13.4 MB 77.5 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 164.9/164.9 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 23.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for unicodecsv: filename=unicodecsv-0.14.1-py3-none-any.whl size=10745 sha256=632d09e5300e28ccbe536b642f1d9ec10f9aeaac0dad838e37df6ce91572015d
  Stored in directory: /root/.cache/pip/wheels/9c/ea/66/8e45247b09052a933eb1a680b7c64802298faba58aac9b346b
Successfully built unicodecsv
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9

In [ ]:
import pythainlp

pythainlp.__version__

'4.0.1'

In [ ]:
from pythainlp import sent_tokenize, word_tokenize

# Read DATA

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!gdown --id 1ebPWQNTKQgzE7GiJr3UgFDJ1UiSBETIQ -O train_attacut.csv

/usr/local/lib/python3.10/dist-packages/gdown/cli.py:121: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1ebPWQNTKQgzE7GiJr3UgFDJ1UiSBETIQ
To: /content/train_attacut.csv
100% 1.31M/1.31M [00:00<00:00, 214MB/s]


In [ ]:
!gdown --id 1KaRm6SINFXyKl7mUx0lpZUfpVxxgWC19 -O test_attacut.csv

/usr/local/lib/python3.10/dist-packages/gdown/cli.py:121: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1KaRm6SINFXyKl7mUx0lpZUfpVxxgWC19
To: /content/test_attacut.csv
100% 546k/546k [00:00<00:00, 191MB/s]


In [ ]:
df_train = pd.read_csv('/content/train_attacut.csv')

In [ ]:
df_test = pd.read_csv('/content/test_attacut.csv')

In [ ]:
X_train = df_train['text_token']
X_test = df_test['text_token']

In [ ]:
y_train = df_train[['toxic','severe_toxic','obscene','threat','insult','identity_hate']]
y_test = df_test[['toxic','severe_toxic','obscene','threat','insult','identity_hate']]

# Sentence Encoder



In [ ]:
!pip3 install tensorflow_text>=2.0.0rc0


In [ ]:
import tensorflow_hub as hub
import numpy as np
import tensorflow_text


In [ ]:
embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder-multilingual/3")


In [ ]:
thai_sentence = ['สวัสดีค้าบทุกๆท่าน']
thai_result = embed(thai_sentence)

In [ ]:
train_text_vector = embed(X_train)
test_text_vector = embed(X_test)

In [ ]:
train_text_vector.shape

TensorShape([7123, 512])

In [ ]:
thai_result

<tf.Tensor: shape=(1, 512), dtype=float32, numpy=
array([[-1.66263580e-02,  4.71503995e-02, -2.79395562e-02,
         2.17557477e-04, -5.88240698e-02,  1.18046310e-02,
         3.72994728e-02, -6.91428664e-04, -8.74652490e-02,
        -4.51822467e-02,  5.07351942e-02, -8.11349368e-04,
         3.05846464e-02,  1.18134944e-02, -2.76617985e-02,
        -1.70718990e-02, -2.09876639e-03, -2.84710433e-02,
         4.03377488e-02, -4.20270227e-02,  1.73879489e-02,
        -4.75095510e-02,  1.10155987e-02, -3.74428406e-02,
         6.75226450e-02, -2.46878136e-02,  2.38603614e-02,
        -2.02788040e-02, -1.93370190e-02, -3.33422311e-02,
        -6.63851481e-03, -1.34660862e-02, -1.15229171e-02,
        -4.38531637e-02,  5.02150469e-02, -1.11891374e-01,
         4.98412624e-02,  2.89980788e-02, -2.44805943e-02,
         1.49655407e-02, -1.04925148e-02,  5.95291406e-02,
        -8.01055953e-02, -4.11083698e-02, -1.15829716e-02,
         2.78788470e-02, -1.79208554e-02,  8.97547603e-03,
      

# USE model

In [ ]:
from keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation
from keras.layers import Bidirectional, GlobalMaxPool1D
from keras.models import Model
from keras import initializers, regularizers, constraints, optimizers, layers

In [ ]:
from keras.preprocessing.text import Tokenizer

In [ ]:
from keras_preprocessing.sequence import pad_sequences

In [ ]:
embed_size = 100 # how big is each word vector
max_features = 20000 # how many unique words to use (i.e num rows in embedding vector)

In [ ]:
maxlen = 512 # max number of words in a comment to use
inp = Input(shape=(maxlen,))

In [ ]:
import tensorflow as tf
import tensorflow_hub as hub

In [ ]:
hub_layer = hub.KerasLayer(embed,input_shape=[],dtype=tf.string, trainable=True)

In [ ]:
# maxlen = 512 # max number of words in a comment to use
# inp = Input(shape=(maxlen,))
# x = Embedding(max_features, embed_size)(inp)
# x = Bidirectional(LSTM(64, return_sequences=True, dropout=0.3, recurrent_dropout=0.3))(x)
# x = GlobalMaxPool1D()(x)
# x = Dense(32, activation="relu")(x)
# x = Dropout(0.3)(x)
# x = Dense(6, activation="sigmoid")(x)
# model = Model(inputs=inp, outputs=x)
# model.compile(optimizer=tf.keras.optimizers.Adam(lr=1e-4), loss='binary_crossentropy', metrics=['accuracy'])
# model.summary()

In [ ]:
model = tf.keras.Sequential()
model.add(hub_layer)
model.add(tf.keras.layers.Dropout(0.3))
model.add(tf.keras.layers.Dense(128, activation='relu'))
model.add(tf.keras.layers.Dropout(0.3))
model.add(tf.keras.layers.Dense(6, activation='sigmoid'))
model.compile(optimizer='adam',loss=tf.keras.losses.BinaryCrossentropy(),metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer (KerasLayer)    (None, 512)               68927232  
                                                                 
 dropout (Dropout)           (None, 512)               0         
                                                                 
 dense (Dense)               (None, 128)               65664     
                                                                 
 dropout_1 (Dropout)         (None, 128)               0         
                                                                 
 dense_1 (Dense)             (None, 6)                 774       
                                                                 
Total params: 68,993,670
Trainable params: 68,993,670
Non-trainable params: 0
_________________________________________________________________


In [ ]:
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard, ModelCheckpoint
# กำหนด callback เพื่อบันทึก checkpoint
checkpoint_path = "/content/drive/MyDrive/training/trainmodel_USE_new.ckpt"
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path,
    save_weights_only=True,
    save_freq=1  # บันทึก checkpoint ทุก 1 epochs
)
# es = EarlyStopping(monitor='val_accuracy', mode='max', patience = 5)
callback_list = [checkpoint_callback]
model.fit(X_train, y_train, batch_size=32, epochs=10, validation_split=0.1, callbacks=[callback_list]);

Epoch 1/10
201/201 [==============================] - 997s 5s/step - loss: 0.2090 - accuracy: 0.5488 - val_loss: 0.1344 - val_accuracy: 0.6157
Epoch 2/10
201/201 [==============================] - 867s 4s/step - loss: 0.1161 - accuracy: 0.5824 - val_loss: 0.1212 - val_accuracy: 0.7532
Epoch 3/10
201/201 [==============================] - 869s 4s/step - loss: 0.0966 - accuracy: 0.6590 - val_loss: 0.1347 - val_accuracy: 0.8387
Epoch 4/10
201/201 [==============================] - 865s 4s/step - loss: 0.0868 - accuracy: 0.6108 - val_loss: 0.1133 - val_accuracy: 0.7826
Epoch 5/10
201/201 [==============================] - 861s 4s/step - loss: 0.0819 - accuracy: 0.5881 - val_loss: 0.1226 - val_accuracy: 0.4432
Epoch 6/10
201/201 [==============================] - 869s 4s/step - loss: 0.0780 - accuracy: 0.5925 - val_loss: 0.1184 - val_accuracy: 0.5175
Epoch 7/10
201/201 [==============================] - 860s 4s/step - loss: 0.0763 - accuracy: 0.6105 - val_loss: 0.1317 - val_accuracy: 0.4769

In [ ]:
import tensorflow as tf
import tensorflow_hub as hub

In [ ]:
hub_layer = hub.KerasLayer(embed,input_shape=[],dtype=tf.string, trainable=True)

In [ ]:
model.save('/content/drive/MyDrive/training/transformer_USE')

In [ ]:
trained_model = tf.keras.models.load_model('/content/drive/MyDrive/training/transformer_USE')

In [ ]:
import numpy as np
from sklearn.metrics import f1_score 

def threshold_tuning(y_true, y_pred_proba, threshold_range=(0.1, 0.9), step=0.1):
    """
    ฟังก์ชั่น threshold_tuning ใช้สำหรับปรับค่า threshold โดยใช้ validation set 
    เพื่อให้ได้ผลลัพธ์ที่ดีที่สุด
    
    :param y_true: numpy array ของค่าจริงของ target variable
    :param y_pred_proba: numpy array ของค่าความน่าจะเป็นที่ได้มาจากโมเดล
    :param threshold_range: tuple ของช่วงของค่า threshold ที่จะทดลอง
    :param step: ค่าของระยะห่างที่ใช้ในการทดสอบ threshold ต่างๆ
    
    :return: tuple ของค่าความแม่นยำที่ดีที่สุดและค่า threshold ที่ใช้
    """
    best_accuracy = 0
    best_threshold = 0

    # ทดสอบ threshold ต่างๆ
    for threshold in np.arange(threshold_range[0], threshold_range[1], step):
        y_pred = (y_pred_proba >= threshold).astype(int)
        accuracy = f1_score(y_true, y_pred, average='micro')

        # ถ้าค่าความแม่นยำสูงกว่า best_accuracy ให้ปรับค่า best_accuracy และ best_threshold
        if accuracy > best_accuracy:
            best_accuracy = accuracy
            best_threshold = threshold

    return best_accuracy, best_threshold
    #     print(accuracy,threshold)


In [ ]:
y_pred_TF = trained_model.predict(X_test)

96/96 [==============================] - 3s 13ms/step


In [ ]:
y_pred_TF

array([[9.4525242e-04, 1.8660929e-06, 1.5408767e-03, 2.3034043e-03,
        6.1639881e-04, 5.9466949e-04],
       [6.1043102e-01, 8.2535623e-04, 2.9524376e-03, 4.4143843e-03,
        9.0770777e-03, 3.4577122e-03],
       [4.7615752e-01, 2.8845293e-03, 1.5468333e-03, 6.3833529e-03,
        6.3548303e-01, 2.1363741e-02],
       ...,
       [9.5973951e-01, 2.3914283e-02, 2.8372444e-02, 6.5761944e-03,
        2.7600756e-02, 1.1318774e-02],
       [1.6459727e-01, 6.7216167e-03, 9.1899276e-01, 4.0150764e-03,
        5.1898560e-03, 3.6450918e-03],
       [3.1050401e-02, 3.1613592e-05, 2.3331938e-03, 1.8523838e-03,
        2.7343091e-03, 8.7379047e-04]], dtype=float32)

In [ ]:
a = y_pred_TF > 0.1

In [ ]:
import numpy
y_pred_threshold01 = numpy.multiply(a, 1)

In [ ]:
list_classes = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']

In [ ]:
threshold_tuning(y_test, y_pred_TF)

(0.7601588756492516, 0.30000000000000004)

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
print(classification_report(y_test, y_pred_threshold01, target_names=list_classes))

               precision    recall  f1-score   support

        toxic       0.68      0.83      0.75       593
 severe_toxic       0.31      0.39      0.34        46
      obscene       0.77      0.97      0.86       694
       threat       0.12      0.08      0.10        48
       insult       0.51      0.64      0.57       238
identity_hate       0.27      0.26      0.27        68

    micro avg       0.66      0.80      0.73      1687
    macro avg       0.44      0.53      0.48      1687
 weighted avg       0.65      0.80      0.72      1687
  samples avg       0.36      0.39      0.36      1687



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
# # โหลดน้ำหนักจาก checkpoint เพื่อเทรนต่อจาก epoch ที่ 1
# latest_checkpoint = tf.train.latest_checkpoint("/content/drive/MyDrive/training/")
# model.load_weights(latest_checkpoint)
# # ฝึกโมเดลต่อเริ่มต้นที่ epoch ที่ 2
# model.fit(X_train, y_train, batch_size=32, epochs=10, initial_epoch=3, validation_split=0.1, callbacks=[callback_list])

In [ ]:
# latest_checkpoint = tf.train.latest_checkpoint("/content/drive/MyDrive/training/")
# model.load_weights(latest_checkpoint)

In [ ]:
Ypred_DF = pd.DataFrame(y_pred_threshold01).rename(columns={0: "toxic_pred", 1: "severe_toxic_pred", 2: "obscene_pred", 3: "threat_pred", 4: "insult_pred",5: "identity_hate_pred"})

In [ ]:
X_test.shape

(3053,)

In [ ]:
y_test.shape

(3053, 6)

In [ ]:
X_test = X_test.reset_index(drop = True)

In [ ]:
Ypred_DF = Ypred_DF.reset_index(drop = True)

In [ ]:
y_test = y_test.reset_index(drop = True)

In [ ]:
DF_compare = pd.concat([X_test, Ypred_DF, y_test], axis = 1)

In [ ]:
DF_compare

,text_token,toxic_pred,severe_toxic_pred,obscene_pred,threat_pred,insult_pred,identity_hate_pred,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,harry ประกาศ เอเชียทัวร์ กุคิด ชุด รอ ละค่า ชุ...,0,0,0,0,0,0,0,0,0,0,0,0
1,ปาก เงียบ โหน กระแส,1,0,0,0,0,0,0,0,0,0,0,0
2,ออกอ่ะยาง อ่ะยาง มั้ง ใจ ท่อง พุท โธ แบ ดยพ่ดก...,1,0,0,0,1,0,0,0,0,0,0,0
3,พ่ก ยิง เรย,0,0,0,0,0,0,0,0,0,0,0,0
4,ราชบุรี ว่าง คิว เย็ด ผม กี่ รอบ สนใจ ทักไลน์p...,0,0,1,0,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3048,ตัวอย่าง ดี คน ชอบ คหสต ทำร้าย สัก พิสูจน์ ดู ...,0,0,0,0,0,0,0,0,0,0,0,0
3049,ดาราจักรรักลานาใจ ดู เศร้า ตอน ฮองเฮา น้อง สาว...,0,0,0,0,0,0,0,0,0,0,0,0
3050,อ๋อ เฉลย ปะ ตัดต่อ อีควาย ชิป ไล่ แปะ รูป ทวิต...,1,0,0,0,0,0,1,1,0,0,0,0
3051,อารมณ์ หวีด ขนส่ง หัวควย,0,0,1,0,0,0,1,0,0,0,0,0


In [ ]:
DF_compare[["text_token", "toxic_pred", "toxic", "severe_toxic_pred", "severe_toxic", "obscene_pred", "obscene", "threat_pred", "threat", "insult_pred", "insult", "identity_hate_pred", "identity_hate"]].to_excel("USE_texttoken.xlsx")  